In [1]:
# Αυτο το κοματι λειτουργει.

from selenium import webdriver
from bs4 import BeautifulSoup
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
import time
import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

url=input('Give the URL please')
tname = input('print the tag name')
a = 'Documento'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv



driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
date_list = []
tag_list = []

stop_date = 20250102
scroll_pause_time = 2.5

#       Poll
driver.get(url)

time.sleep(2)
driver.maximize_window()
time.sleep(1)
disagree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

flag = True
inside_flag = True
def scroll(driver, timeout):
    global flag
    global stop_date
    global inside_flag
    global tag_list
    global date_list
    scroll_pause_time = timeout
    nr_of_scrollings = 2
    changable_nr_of_scrollings = nr_of_scrollings
    #inside_flag = True

    # this function checks the date on the scrolling page. The all articles page.
    # I will try to make it dual usable. To stop the scrolling, but also after the scrolling..
    def date_check_fun():
        global inside_flag
        #inside_flag = True
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'lxml')

        article_place = soup.find('div' , class_ = "posts-list")
        dates = article_place.find_all('div' , class_ = "item-meta")
        date = dates[-1].get_text()
        date = date.split(",")
        date = date[0].replace("\n" , "")
        #print(date)
        dates = date.split('/')
        #print(dates)
        dates.reverse()
        #print(dates)
        date = "".join(dates)
        date = int(date)

        if date < stop_date :
            print('the date is smaller. Programm Must STOP')
            inside_flag = False
            return inside_flag
        

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while inside_flag:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            #flag=False
            break
        last_height = new_height
        # i want every some scroll downs to check the date
        changable_nr_of_scrollings = changable_nr_of_scrollings - 1
        #nr_of_scrollings = nr_of_scrollings - 1
        if changable_nr_of_scrollings == 0:
            date_check_fun()
            #print('on the outer/mother function the inside_flag is  ' , inside_flag)
            #nr_of_scrollings = 2 
            changable_nr_of_scrollings = nr_of_scrollings

#   THIS PART OF CODE I WORK now. The apo epanw functions properly.

    def tag_function(inside):
        global tag_list
        inside_tag_list = []

        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')
        tag_section = soup.find('div' , class_ = 'tags-container')
        
        try:
            for i in tag_section("a"):
                #b = i.find('a')
                tag = i.get_text()
                inside_tag_list.append(tag)
    
            # Now, i will append inside the tag_list also the outer tag (for example: ΚΟΣΜΟΣ)
            outer_tag = soup.find('div' , class_ = 'meta')
            outer_tag = outer_tag.find('a')
            outer_tag = outer_tag.get_text()
            inside_tag_list.append(outer_tag)
            
            tag_list.append(inside_tag_list)
        except:
            print('i have a problem. the problematic url is ' , inside)


    #inside_flag = True
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')

    article_place = soup.find('div' , class_ = "posts-list")
    articles = article_place.find_all('div' , class_ = 'meta-container')
    
    for article in articles:
        date = article.find('div' , class_ = "item-meta")
        date = date.get_text()
        date = date.split(",")
        date = date[0].replace("\n" , "")
        # print(date)
        dates = date.split('/')
        # print(dates)
        dates.reverse()
        # print(dates)
        date = "".join(dates)
        date = int(date)

        if date < stop_date :
            print('the date is smaller. I need to terminate the whole programm')
            break
        else:
            article_url = article.find('h2' , class_ = 'entry-title')
            article_url = article_url.find('a')
            article_url = article_url.get('href')
            # i have taken the url. Pleon, proxwraw opws kai stin kathimerini..
            date_list.append(date)
            # i need to check if the url is fine:
            https = "https://www.documentonews.gr"
            if article_url.find(https)<0:
                article_url = https + article_url
            
            tag_function(article_url)
            
        
scroll(driver,scroll_pause_time)



import pandas as pd
# df_costliness = pd.DataFrame(tag_list,date_list)
# df_costliness.to_csv('costliness.csv')

#print(tag_list)
#print(date_list)

new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        #if tag_list[i][j] not in new_tag_list:
       new_tag_list.append(tag_list[i][j])
#print(new_tag_list)

res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

resultList = new_list = list(map(list, res.items()))
# printing the resultant list of a dictionary key-values
print(resultList)

df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)


#print(tag_list)
print(len(tag_list))
print(len(date_list))
print('the last date collected is ' , date_list[-1])


# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=date_list
ttag_list = tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)

n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    for tag in i:
        inside=[]
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)


print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


{'METRON ANALYSIS': 3, 'ΔΗΜΟΣΚΟΠΗΣΗ': 20, 'ΠΟΛΙΤΙΚΗ': 18, 'ΚΥΡΙΑΚΟΣ ΜΗΤΣΟΤΑΚΗΣ': 5, 'ΝΔ': 3, 'MRB': 2, 'ΑΛΕΞΗΣ ΤΣΙΠΡΑΣ': 1, 'MARC': 1, 'PODCAST': 1, 'ΝΕΑ ΔΗΜΟΚΡΑΤΙΑ': 2, 'ΠΑΣΟΚ': 2, 'ΣΥΡΙΖΑ': 1, 'ΚΟΣΜΟΣ': 2, 'ΚΥΒΕΡΝΗΣΗ ΜΗΤΣΟΤΑΚΗ': 2, 'ΤΕΜΠΗ': 4, 'ΕΥΡΩΠΑΪΚΗ ΕΝΩΣΗ': 1, 'ΚΑΝΑΔΑΣ': 1, 'ΕΓΚΛΗΜΑ ΤΕΜΠΗ': 8, 'ALCO': 1, 'PROTAGON': 1, 'ΣΥΛΛΑΛΗΤΗΡΙΑ': 1, 'ΓΙΩΡΓΟΣ ΚΥΡΤΣΟΣ': 1}
[['METRON ANALYSIS', 3], ['ΔΗΜΟΣΚΟΠΗΣΗ', 20], ['ΠΟΛΙΤΙΚΗ', 18], ['ΚΥΡΙΑΚΟΣ ΜΗΤΣΟΤΑΚΗΣ', 5], ['ΝΔ', 3], ['MRB', 2], ['ΑΛΕΞΗΣ ΤΣΙΠΡΑΣ', 1], ['MARC', 1], ['PODCAST', 1], ['ΝΕΑ ΔΗΜΟΚΡΑΤΙΑ', 2], ['ΠΑΣΟΚ', 2], ['ΣΥΡΙΖΑ', 1], ['ΚΟΣΜΟΣ', 2], ['ΚΥΒΕΡΝΗΣΗ ΜΗΤΣΟΤΑΚΗ', 2], ['ΤΕΜΠΗ', 4], ['ΕΥΡΩΠΑΪΚΗ ΕΝΩΣΗ', 1], ['ΚΑΝΑΔΑΣ', 1], ['ΕΓΚΛΗΜΑ ΤΕΜΠΗ', 8], ['ALCO', 1], ['PROTAGON', 1], ['ΣΥΛΛΑΛΗΤΗΡΙΑ', 1], ['ΓΙΩΡΓΟΣ ΚΥΡΤΣΟΣ', 1]]
20
20
the last date collected is  20250226
[20250522, 20250514, 20250423, 20250410, 20250407, 20250402, 20250328, 20250325, 20250321, 20250320, 20250320, 20250319, 20250318, 20250313, 20250307, 20250306, 20250306, 2025